### Preparar ambiente

In [1]:
import sys
!{sys.executable} -m pip install PyAthena

In [2]:
import sys
!{sys.executable} -m pip install pytz

### Conectar con pyathena

In [3]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

### Eligiendo pais y periodo

In [4]:
paises = ['BO','EC','CL','VE','AR','CR','DO','SV','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR']
len(paises)

22

In [5]:
pais = 'BR' # 'PE' 'BO' 'PY' 'UY'

In [6]:
fecha_inicio = '09-01-2020'
fecha_final = '09-30-2020'
fechas = pd.date_range(fecha_inicio, fecha_final)

### Union de tablas usuarios según sub regiones

In [7]:
historico_usuarios_regiones = f'historico_usuarios_regiones_{pais}'
start_time = time.time()
for i,fecha in enumerate(fechas):
    time_fecha = time.time()
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    print(codigo_fecha)
    tabla_usuarios_fecha = f'historico_usuarios_adms_{pais}_{codigo_fecha}'
    if i == 0 and (day == '01' and fecha.month == 2):
        query = f'''
                    CREATE TABLE {historico_usuarios_regiones}
                    WITH ( external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-usuarios/Historico_usuarios_regiones/{historico_usuarios_regiones}', 
                        format = 'PARQUET', 
                        parquet_compression = 'SNAPPY') AS
                    SELECT *
                    FROM {tabla_usuarios_fecha}
                '''
    else:
        query = f'''
                    INSERT INTO {historico_usuarios_regiones} SELECT * FROM {tabla_usuarios_fecha} 
                '''
    cursor.execute(query)
    tiempo = (time.time()-time_fecha)/60
    print(f"{codigo_fecha} insertada, {int(tiempo)} minutos")
print((time.time()-start_time)/60)

901
901 insertada, 0 minutos
902
902 insertada, 0 minutos
903
903 insertada, 0 minutos
904
904 insertada, 0 minutos
905
905 insertada, 0 minutos
906
906 insertada, 0 minutos
907
907 insertada, 0 minutos
908
908 insertada, 0 minutos
909
909 insertada, 0 minutos
910
910 insertada, 0 minutos
911
911 insertada, 0 minutos
912
912 insertada, 0 minutos
913
913 insertada, 0 minutos
914
914 insertada, 0 minutos
915
915 insertada, 0 minutos
916
916 insertada, 0 minutos
917
917 insertada, 0 minutos
918
918 insertada, 0 minutos
919
919 insertada, 0 minutos
920
920 insertada, 0 minutos
921
921 insertada, 0 minutos
922
922 insertada, 0 minutos
923
923 insertada, 0 minutos
924
924 insertada, 0 minutos
925
925 insertada, 0 minutos
926
926 insertada, 0 minutos
927
927 insertada, 0 minutos
928
928 insertada, 0 minutos
929
929 insertada, 0 minutos
930
930 insertada, 0 minutos
3.7844333132108052


In [8]:
query = f'''MSCK REPAIR TABLE {historico_usuarios_regiones}'''
cursor.execute(query)

tabla_datos_pais = f"historico_usuarios_regiones_{pais}"
query = f"DROP TABLE {tabla_datos_pais}"
cursor.execute(query)

print('Tablas eliminadas')